<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/277_GCO_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Governance & Compliance Orchestrator Agent

In [ ]:
# ============================================================================
# Governance & Compliance Orchestrator Agent
# ============================================================================

class GovernanceComplianceOrchestratorState(TypedDict, total=False):
    """State for Governance & Compliance Orchestrator Agent"""

    # Input fields
    agent_name: Optional[str]              # Specific agent to analyze (None = analyze all)
    time_window_days: Optional[int]        # Time window for analysis (None = use default)

    # Goal & Planning fields (MVP: Fixed goal, template-based plan)
    goal: Dict[str, Any]                   # Goal definition (from goal_node)
    plan: List[Dict[str, Any]]            # Execution plan (from planning_node)

    # Data Ingestion
    agent_action_logs: List[Dict[str, Any]]  # Loaded agent action log events
    # Structure per event:
    # {
    #   "event_id": "evt_0001",
    #   "timestamp": "2026-01-02T09:05:12Z",
    #   "agent_name": "SalesEnablementAgent",
    #   "action_type": "pricing_recommendation",
    #   "input_data": {...},
    #   "output": {...},
    #   "model": "gpt-4.1",
    #   "confidence_score": 0.78,
    #   "human_in_the_loop": false,
    #   "data_sources": ["CRM", "SalesForecast_v1"]
    # }

    policy_rules: List[Dict[str, Any]]     # Loaded policy rules
    # Structure per policy:
    # {
    #   "policy_id": "EU_HIGH_RISK_REQUIRES_APPROVAL",
    #   "description": "...",
    #   "conditions": {...},
    #   "required_action": "human_approval",
    #   "severity": "high"
    # }

    bias_signals: List[Dict[str, Any]]    # Loaded bias detection signals
    # Structure per signal:
    # {
    #   "signal_id": "bias_001",
    #   "agent_name": "HRDecisionAgent",
    #   "decision_type": "hiring_decision",
    #   "protected_attribute": "gender",
    #   "groups": [...],
    #   "delta": 0.31,
    #   "threshold": 0.20,
    #   "risk_level": "high",
    #   "recommended_action": "..."
    # }

    drift_signals: List[Dict[str, Any]]   # Loaded drift and degradation signals
    # Structure per signal:
    # {
    #   "signal_id": "drift_001",
    #   "agent_name": "CustomerSupportAgent",
    #   "model": "gpt-4.1",
    #   "metric": "hallucination_rate",
    #   "previous_average": 0.03,
    #   "current_average": 0.11,
    #   "threshold": 0.08,
    #   "delta": 0.08,
    #   "risk_level": "high",
    #   "detected_at": "2026-01-02T12:55:00Z",
    #   "recommended_action": "..."
    # }

    # Data Lookups (for fast access)
    policy_lookup: Dict[str, Dict[str, Any]]  # policy_id -> policy dict
    events_lookup: Dict[str, Dict[str, Any]]  # event_id -> event dict

    # Policy Evaluation
    policy_evaluations: List[Dict[str, Any]]  # Policy matches per event
    # Structure per evaluation:
    # {
    #   "event_id": "evt_0002",
    #   "policy_id": "EU_HIGH_RISK_REQUIRES_APPROVAL",
    #   "matched": true,
    #   "violation": true,
    #   "severity": "high",
    #   "required_action": "human_approval",
    #   "reason": "EU region + confidence < 0.6 + no human approval"
    # }

    # Risk Assessment
    compliance_events: List[Dict[str, Any]]  # Generated compliance events (violations)
    # Structure per event:
    # {
    #   "compliance_event_id": "cmp_0001",
    #   "event_id": "evt_0002",
    #   "risk_type": "policy_violation",
    #   "policy_id": "EU_HIGH_RISK_REQUIRES_APPROVAL",
    #   "severity": "high",
    #   "status": "open",
    #   "recommended_action": "Escalate to compliance officer",
    #   "timestamp": "2026-01-02T14:32:12Z"
    # }

    risk_scores: Dict[str, Any]           # Risk scores per agent/event
    # Structure:
    # {
    #   "agent_scores": {
    #     "SalesEnablementAgent": {
    #       "total_violations": 3,
    #       "high_severity_count": 2,
    #       "risk_score": 0.75
    #     }
    #   },
    #   "overall_risk_score": 0.68
    # }

    # Prioritization
    prioritized_issues: List[Dict[str, Any]]  # Prioritized compliance issues
    # Structure per issue:
    # {
    #   "compliance_event_id": "cmp_0001",
    #   "priority_score": 85.5,
    #   "severity": "high",
    #   "urgency": "high",
    #   "agent_name": "SalesEnablementAgent"
    # }

    # Summary
    summary: Dict[str, Any]               # Overall summary statistics
    # Structure:
    # {
    #   "total_events_analyzed": 36,
    #   "total_violations": 8,
    #   "high_severity_count": 4,
    #   "bias_signals_count": 4,
    #   "drift_signals_count": 5,
    #   "agents_affected": ["SalesEnablementAgent", "HRDecisionAgent"]
    # }

    # Output
    audit_report: str                     # Generated audit report (markdown)
    report_file_path: Optional[str]       # Path to saved report file

    # Metadata
    errors: List[str]                     # Any errors encountered
    processing_time: Optional[float]      # Time taken to process


@dataclass
class GovernanceComplianceOrchestratorConfig:
    """Configuration for Governance & Compliance Orchestrator Agent"""

    # LLM Settings
    llm_model: str = os.getenv("LLM_MODEL", "gpt-4o-mini")
    temperature: float = 0.3

    # Data file paths
    data_dir: str = "agents/data"
    agent_logs_files: List[str] = field(default_factory=lambda: [
        "agent_action_logs_batch_1.json",
        "agent_action_logs_batch_2.json",
        "agent_action_logs_batch_3.json"
    ])
    policy_rules_file: str = "policy_rules.json"
    bias_signals_file: str = "bias_signals.json"
    drift_signals_file: str = "drift_and_degradation_signals.json"

    # Report settings
    reports_dir: str = "output/governance_compliance_reports"

    # Policy Evaluation Settings
    default_time_window_days: int = 30    # Default time window for analysis

    # Risk Scoring Settings
    severity_weights: Dict[str, float] = field(default_factory=lambda: {
        "critical": 1.0,
        "high": 0.75,
        "medium": 0.50,
        "low": 0.25
    })

    # Bias Detection Settings
    bias_delta_threshold: float = 0.20    # Minimum delta to flag as bias
    bias_risk_levels: Dict[str, float] = field(default_factory=lambda: {
        "critical": 0.50,
        "high": 0.30,
        "medium": 0.20,
        "low": 0.10
    })

    # Drift Detection Settings
    drift_threshold_multiplier: float = 1.2  # Threshold multiplier for drift detection

    # Priority Scoring Weights (CEO-friendly transparency)
    priority_scoring_weights: Dict[str, float] = field(default_factory=lambda: {
        "severity": 0.40,
        "urgency": 0.30,
        "impact": 0.20,
        "frequency": 0.10
    })

    # Toolshed Integration
    enable_prioritization: bool = True     # Use toolshed.prioritization
    enable_reporting: bool = True          # Use toolshed.reporting

    # LLM Enhancement (Optional - Phase 8)
    enable_llm_explanations: bool = False  # Enable LLM-generated violation explanations
    llm_explanation_max_events: int = 5    # Max events to generate LLM explanations for (cost control)

